In [ ]:
## 输入：model, optimizer, baseline, lr_scheduler, epoch, val_dataset, problem, tb_logger, opt

# def train_epoch(model, optimizer, baseline, lr_scheduler, epoch, val_dataset, problem, tb_logger, opt):
#     print(f'Epoch {epoch}, lr= {optimizer.param_groups[0]["lr"]}')


#!/usr/bin/env python

from pointer_network import PointerNetwork
from attention_model import AttentionModel
from critic import CriticNetwork
from options import get_options
import pprint as pp
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import math
import tqdm
import torch.nn.Functional as F

opts = get_options()
model = AttentionModel(opts)
optimizer = optim.Adam(model.parameters(), lr=opts.lr)



def calculate_angle(point1, point2):
    # 计算两点之间的角度
    delta_x = point2[0] - point1[0]
    delta_y = point2[1] - point1[1]
    angle = math.atan2(delta_y, delta_x) * 180 / math.pi
    return angle

def cal_angle_and_mask_matrix(tasks):
    '''Calculate angle between tasks and mask matrix.
    Args:
        tasks (np.array): (batch_size, n_node, 2)
    Returns:
        angle (np.array): (batch_size, n_node, n_node)
        '''
    n_epoch, n_batch_per_epoch,batch_size, n_node, _ = tasks.shape
    angle = np.zeros((n_epoch, n_batch_per_epoch,batch_size, n_node, n_node))
    mask_matrix = np.zeros((n_epoch, n_batch_per_epoch,batch_size, n_node, n_node))
    for m in range(n_epoch):
        for n in range(n_batch_per_epoch):
            for i in range(batch_size):
                for j in range(n_node):
                    for k in range(n_node):
                        angle[m,n,i,j,k] = calculate_angle(tasks[m,n,i,j], tasks[m,n,i,k])
                        
                        if -30 < angle[m,n,i,j,k] < 30:
                            mask_matrix[m,n,i,j,k] = 1
    
    return angle, mask_matrix


for i in range(opts.epoch):
    print(f'Epoch {i}, lr= {optimizer.param_groups[0]["lr"]}')

    ## generate data and cal mask matrix
    batch_size = 10
    n_node = 20
    # n_epoch=2 
    n_batch_per_epoch = 100

    ##TODO：随机任务个数，pytorch dataloader
    training_dataset = np.random.uniform(size=(n_batch_per_epoch,batch_size, n_node, 2)).astype(np.float32)
    training_dataloader = DataLoader(training_dataset, batch_size=opts.batch_size, num_workers=1)
    angle1, mask_matrix = cal_angle_and_mask_matrix(training_dataset)
    ## train
    model.train()
    for batch_id, batch in enumerate(tqdm(training_dataloader)):
        print(f'Start train batch {batch_id+1}')
        ## forward
        ## action mask
        ## cost = -length(action)
        cost, log_likelihood = model(batch)

        v = CriticNetwork(batch)
        bl_val, bl_loss = v.detach(), F.mse_loss(v, cost.detach())

        # Calculate loss
        reinforce_loss = ((cost - bl_val) * log_likelihood).mean()
        loss = reinforce_loss + bl_loss

        ## backward
        optimizer.zero_grad()
        loss.backward()
    

